In [2]:
import pandas as pd

In [3]:
# read in data csv/frames

circuits_df = pd.read_csv("data/circuits.csv")
constructor_results_df = pd.read_csv("data/constructor_results.csv")
constructors_df = pd.read_csv("data/constructors.csv")
driver_standings_df = pd.read_csv("data/driver_standings.csv")
drivers_df = pd.read_csv("data/drivers.csv")
lap_times_df = pd.read_csv("data/lap_times.csv")
pit_stops_df = pd.read_csv("data/pit_stops.csv")
qualifying_df = pd.read_csv("data/qualifying.csv")
races_df = pd.read_csv("data/races.csv")
results_df = pd.read_csv("data/results.csv")
seasons_df = pd.read_csv("data/seasons.csv")
sprint_res_df = pd.read_csv("data/sprint_results.csv")
status_df = pd.read_csv("data/status.csv")
tyre_data_df = pd.read_csv("data/tyre_data.csv")
races_with_weather_df = pd.read_csv("data/races_with_weather.csv")
races_with_dummies_df = pd.read_csv("data/races_with_dummies.csv")
tyre_data_new_df = pd.read_csv("data/tyre_data_new.csv")
tyre_data_ID_df = pd.read_csv("data/tyre_data_drRef.csv")

In [4]:
tyre_data_ID_df.columns

Index(['Unnamed: 0', 'Driver', 'GP', 'Tyres', 'From', 'To', '#Laps', 'year',
       'location', 'raceId', 'driverId', 'driverRef'],
      dtype='object')

In [5]:
# change spanish words to english / have to do it before the processing - should be better

tyres_mapping = {
    'Hiper Blando Usado': 'Hyper Soft',
    'Hiper Blando Nuevo': 'Hyper Soft',
    'Lluvia Extrema Usado': 'Full Wet'
}

# Use the replace() method to update the values in the "tyres" column
tyre_data_ID_df['Tyres'] = tyre_data_ID_df['Tyres'].replace(tyres_mapping)

In [8]:
grouped = tyre_data_ID_df.groupby('year')

# Create a dictionary to store unique values for each year
unique_values_by_year = {}

# Iterate through each group (year) and get unique values for the "GP" column
for year, group in grouped:
    unique_values = group['GP'].unique()
    unique_values_by_year[year] = unique_values

# Print or access unique values for each year
for year, values in unique_values_by_year.items():
    print(f'Year {year}: {values}')

Year 2012: ['Australia' 'Malaysia' 'China' 'Bahrain' 'Spain' 'Monaco' 'Canada'
 'Europa' 'Great Britain' 'Germany' 'Hungary' 'Belgium' 'Italy'
 'Singapore' 'Japan' 'Korea' 'India' 'Abu Dhabi' 'United States' 'Brazil']
Year 2013: ['Australia' 'Malaysia' 'China' 'Bahrain' 'Spain' 'Monaco' 'Canada'
 'Great Britain' 'Germany' 'Hungary' 'Belgium' 'Italy' 'Singapore' 'Korea'
 'Japan' 'India' 'Abu Dhabi' 'United States' 'Brazil']
Year 2014: ['Australia' 'Malaysia' 'Bahrain' 'China' 'Spain' 'Monaco' 'Canada'
 'Austria' 'Great Britain' 'Germany' 'Hungary' 'Belgium' 'Italy'
 'Singapore' 'Japan' 'Russia' 'United States' 'Brazil' 'Abu Dhabi']
Year 2015: ['Australia' 'Malaysia' 'China' 'Bahrain' 'Spain' 'Monaco' 'Canada'
 'Austria' 'Great Britain' 'Hungary' 'Belgium' 'Italy' 'Singapore' 'Japan'
 'Russia' 'United States' 'Mexico' 'Brazil' 'Abu Dhabi']
Year 2016: ['Australia' 'Bahrain' 'China' 'Russia' 'Spain' 'Monaco' 'Canada' 'Europe'
 'Austria' 'Great Britain' 'Hungary' 'Germany' 'Belgium' 'Italy'

In [ ]:
#If you want to know when there were pit stops in addition to the total laps driven on each tire type, you can modify the code to include information about pit stops. You can achieve this by keeping track of the start and end lap numbers for each tire stint. Here's an updated version of your code that includes information about pit stops:
# best so far

import pandas as pd

# Load your tyre_data_ID_df DataFrame here

# Sort the data by driverId, raceId, and lap number
tyre_data_ID_df.sort_values(by=['driverId', 'raceId', '#Laps'], inplace=True)

# Initialize variables to keep track of pit stops
current_tyre = None
current_tyre_start_lap = None
pit_stops = []

# Iterate through the rows and record pit stops
for index, row in tyre_data_ID_df.iterrows():
    if current_tyre is None:
        current_tyre = row['Tyres']
        current_tyre_start_lap = row['#Laps']
    elif current_tyre != row['Tyres']:
        pit_stops.append({
            'driverId': row['driverId'],
            'raceId': row['raceId'],
            'Driver': row['Driver'],
            'GP': row['GP'],
            'year': row['year'],
            'location': row['location'],
            'driverRef': row['driverRef'],
            'Tyres': current_tyre,
            'Start Lap': current_tyre_start_lap,
            'End Lap': row['#Laps'] - 1
        })
        current_tyre = row['Tyres']
        current_tyre_start_lap = row['#Laps']

# Convert pit_stops list to a DataFrame
pit_stops_df = pd.DataFrame(pit_stops)

# Group by driverId, raceId, Tyres, and other shared columns, and sum the lap numbers
consolidated_data = tyre_data_ID_df.groupby(['driverId', 'raceId', 'Driver', 'GP', 'year', 'location', 'driverRef', 'Tyres'], as_index=False)['#Laps'].sum()

# Fill missing values with 0 for laps when a driver did not use a particular tire type
consolidated_data.fillna(0, inplace=True)

# Rename the columns for clarity
consolidated_data.rename(columns={'#Laps': 'Total Laps'}, inplace=True)

# Print the consolidated data with pit stops
print(consolidated_data)


consolidated_data.to_csv("data/consolidated_tyre_data_with_pit_stop.csv")



      driverId  raceId          Driver           GP  year      location   
0          1.0     860  Lewis Hamilton    Australia  2012      Adelaide  \
1          1.0     860  Lewis Hamilton    Australia  2012      Adelaide   
2          1.0     861  Lewis Hamilton     Malaysia  2012  Kuala Lumpur   
3          1.0     861  Lewis Hamilton     Malaysia  2012  Kuala Lumpur   
4          1.0     861  Lewis Hamilton     Malaysia  2012  Kuala Lumpur   
...        ...     ...             ...          ...   ...           ...   
8970     858.0    1109  Logan Sargeant      Hungary  2023      Budapest   
8971     858.0    1109  Logan Sargeant      Hungary  2023      Budapest   
8972     858.0    1110  Logan Sargeant      Belgium  2023      Brussels   
8973     858.0    1110  Logan Sargeant      Belgium  2023      Brussels   
8974     858.0    1111  Logan Sargeant  Netherlands  2023     Zandvoort   

     driverRef             Tyres  Total Laps  
0     hamilton        Medium New          41  
1    

In [ ]:
# SOLUTION FOR own column for each tyre value with rounds driven

# Load your tyre_data_ID_df DataFrame here

# Sort the data by driverId, raceId, and lap number
tyre_data_ID_df.sort_values(by=['driverId', 'raceId', '#Laps'], inplace=True)

# Initialize variables to keep track of pit stops
current_tyre = None
current_tyre_start_lap = None
pit_stops = []

# Iterate through the rows and record pit stops
for index, row in tyre_data_ID_df.iterrows():
    if current_tyre is None:
        current_tyre = row['Tyres']
        current_tyre_start_lap = row['#Laps']
    elif current_tyre != row['Tyres']:
        pit_stops.append({
            'driverId': row['driverId'],
            'raceId': row['raceId'],
            'Driver': row['Driver'],
            'GP': row['GP'],
            'year': row['year'],
            'location': row['location'],
            'driverRef': row['driverRef'],
            'Tyres': current_tyre,
            'Start Lap': current_tyre_start_lap,
            'End Lap': row['#Laps'] - 1
        })
        current_tyre = row['Tyres']
        current_tyre_start_lap = row['#Laps']

# Convert pit_stops list to a DataFrame
pit_stops_df = pd.DataFrame(pit_stops)

# Group by driverId, raceId, Tyres, and other shared columns, and sum the lap numbers
consolidated_data = tyre_data_ID_df.groupby(['driverId', 'raceId', 'Driver', 'GP', 'year', 'location', 'driverRef', 'Tyres'], as_index=False)['#Laps'].sum()

# Fill missing values with 0 for laps when a driver did not use a particular tire type
consolidated_data.fillna(0, inplace=True)

# Pivot the data to have separate columns for each tire type with total laps per race
pivoted_data = consolidated_data.pivot_table(index=['driverId', 'raceId', 'Driver', 'GP', 'year', 'location', 'driverRef'], columns='Tyres', values='#Laps', fill_value=0).reset_index()

# Rename the columns for clarity
pivoted_data.rename_axis(None, axis=1, inplace=True)

# Print the consolidated data with pit stops
print(pivoted_data)

pivoted_data.to_csv("data/tyre_data_with_total_laps_per_race.csv", index=False)


      driverId  raceId          Driver           GP  year      location   
0          1.0     860  Lewis Hamilton    Australia  2012      Adelaide  \
1          1.0     861  Lewis Hamilton     Malaysia  2012  Kuala Lumpur   
2          1.0     862  Lewis Hamilton        China  2012      Shanghai   
3          1.0     863  Lewis Hamilton      Bahrain  2012        Sakhir   
4          1.0     864  Lewis Hamilton        Spain  2012     Barcelona   
...        ...     ...             ...          ...   ...           ...   
4030     858.0    1105  Logan Sargeant        Spain  2023     Barcelona   
4031     858.0    1106  Logan Sargeant       Canada  2023        Quebec   
4032     858.0    1109  Logan Sargeant      Hungary  2023      Budapest   
4033     858.0    1110  Logan Sargeant      Belgium  2023      Brussels   
4034     858.0    1111  Logan Sargeant  Netherlands  2023     Zandvoort   

     driverRef  Full Wet  Hard New  Hard Used  ...  Intermediate Used   
0     hamilton         0  

In [ ]:
pivoted_data.columns

Index(['driverId', 'raceId', 'Driver', 'GP', 'year', 'location', 'driverRef',
       'Full Wet', 'Hard New', 'Hard Used', 'Hyper Soft', 'Intermediate New',
       'Intermediate Used', 'Medium New', 'Medium Used', 'Soft New',
       'Soft Used', 'Supersoft New', 'Supersoft Used', 'Ultrasoft New',
       'Ultrasoft Used', 'Wet New'],
      dtype='object')

In [ ]:
pivoted_data.raceId.nunique()

188

In [ ]:
pivoted_data.year.nunique()

12

ca. 15 races per year

In [ ]:
pivoted_data.to_csv("data/pivoted_tyre_data_df.csv")

based on df above we could add info from the pit stop data frame and maybe devide race into parts separated by the pit stops: e.g. part1 soft, part 



In [ ]:
import pandas as pd

# Assuming you have the dataset loaded into a DataFrame named 'df'
# If not, you can read the CSV file using df = pd.read_csv('your_file.csv')

# Sort the DataFrame by 'raceId' and 'driverId' to ensure sequential data
df = df.sort_values(by=['raceId', 'driverId'])

# Create a new column 'pit_stop' to identify when tyres are changed
df['pit_stop'] = (df['Tyres'] != df['Tyres'].shift(1)).astype(int)

# Calculate the cumulative sum of pit stops to get the pit stop number
df['pit_stop_number'] = df.groupby(['raceId', 'driverId'])['pit_stop'].cumsum()

# Calculate the lap number (round) where each pit stop took place
df['lap_when_pit_stop'] = df.groupby(['raceId', 'driverId', 'pit_stop_number']).cumcount() + 1

# Group by 'raceId', 'driverId', 'Tyres' and sum the laps for each tyre value
result = df.groupby(['raceId', 'driverId', 'Tyres'])[['Total Laps', 'lap_when_pit_stop']].max().reset_index()

# Pivot the table to get each tyre value as a separate column
result = result.pivot(index=['raceId', 'driverId'], columns='Tyres', values=['Total Laps', 'lap_when_pit_stop']).reset_index()

# Fill NaN values with 0 for races where a specific tyre wasn't used
result = result.fillna(0)

# Rename the columns for clarity
result.columns = ['raceId', 'driverId'] + [f'Total Laps - {col[0]}' for col in result.columns if col[0] != ('raceId', 'driverId')] + [f'Lap when Pit Stop - {col[1]}' for col in result.columns if col[0] == 'lap_when_pit_stop']

# Merge the pit stop information into the result DataFrame
result = pd.merge(result, df[['raceId', 'driverId', 'pit_stop_number']], on=['raceId', 'driverId'], how='left')

# Fill NaN pit stop numbers with 0
result['pit_stop_number'] = result['pit_stop_number'].fillna(0).astype(int)

# Print the result DataFrame
print(result)


ValueError: Length mismatch: Expected axis has 12 elements, new values have 19 elements

In [ ]:
import pandas as pd

# Load your tyre_data_ID_df DataFrame here

# Sort the data by driverId, raceId, and lap number
tyre_data_ID_df.sort_values(by=['driverId', 'raceId', '#Laps'], inplace=True)

# Initialize variables to keep track of pit stops
current_tyre = None
current_tyre_start_lap = None
pit_stops = []

# Create a dictionary to store consolidated data
consolidated_data = {
    'driverId': [],
    'raceId': [],
    'Driver': [],
    'GP': [],
    'year': [],
    'location': [],
    'driverRef': [],
}

# Initialize columns for each tire type
unique_tyres = tyre_data_ID_df['Tyres'].unique()
for tyre_type in unique_tyres:
    consolidated_data[tyre_type + ' Start Lap'] = []
    consolidated_data[tyre_type + ' End Lap'] = []

# Iterate through the rows and record pit stops and consolidate tire data
for index, row in tyre_data_ID_df.iterrows():
    if current_tyre is None:
        current_tyre = row['Tyres']
        current_tyre_start_lap = row['#Laps']
    elif current_tyre != row['Tyres']:
        pit_stops.append({
            'driverId': row['driverId'],
            'raceId': row['raceId'],
            'Driver': row['Driver'],
            'GP': row['GP'],
            'year': row['year'],
            'location': row['location'],
            'driverRef': row['driverRef'],
            'Tyres': current_tyre,
            'Start Lap': current_tyre_start_lap,
            'End Lap': row['#Laps'] - 1
        })
        current_tyre = row['Tyres']
        current_tyre_start_lap = row['#Laps']

    # Add tire data to consolidated_data dictionary
    consolidated_data[row['Tyres'] + ' Start Lap'].append(current_tyre_start_lap)
    consolidated_data[row['Tyres'] + ' End Lap'].append(row['#Laps'] - 1)

    # Add other shared data to consolidated_data dictionary
    for key in ['driverId', 'raceId', 'Driver', 'GP', 'year', 'location', 'driverRef']:
        consolidated_data[key].append(row[key])

# Convert pit_stops list to a DataFrame
pit_stops_df = pd.DataFrame(pit_stops)

# Convert consolidated_data dictionary to a DataFrame
consolidated_data_df = pd.DataFrame(consolidated_data)

# Print the consolidated data with pit stops
print(consolidated_data_df)

# Save the consolidated DataFrame to a CSV file
consolidated_data_df.to_csv("data/consolidated_tyre_data_with_pit_stop.csv")


ValueError: All arrays must be of the same length

In [ ]:

# Sort the data by driverId, raceId, and lap number
tyre_data_ID_df.sort_values(by=['driverId', 'raceId', '#Laps'], inplace=True)

# Initialize variables to keep track of tire information
current_tyre = None
current_tyre_start_lap = None
tire_info = []

# Iterate through the rows and record tire information
for index, row in tyre_data_ID_df.iterrows():
    if current_tyre is None:
        current_tyre = row['Tyres']
        current_tyre_start_lap = row['#Laps']
    elif current_tyre != row['Tyres']:
        tire_info.append(f"{current_tyre} (Laps {current_tyre_start_lap}-{row['#Laps'] - 1})")
        current_tyre = row['Tyres']
        current_tyre_start_lap = row['#Laps']

# Convert tire_info list to a single string
tire_info_str = ', '.join(tire_info)

# Group by driverId, raceId, and other shared columns, and sum the lap numbers
consolidated_data = tyre_data_ID_df.groupby(['driverId', 'raceId', 'Driver', 'GP', 'year', 'location', 'driverRef'], as_index=False)['#Laps'].sum()

# Fill missing values with 0 for laps when a driver did not use a particular tire type
consolidated_data.fillna(0, inplace=True)

# Add a column for tire information
consolidated_data['Tire Info'] = tire_info_str

# Rename the columns for clarity
consolidated_data.rename(columns={'#Laps': 'Total Laps'}, inplace=True)

# Print the consolidated data with tire information
print(consolidated_data)

# Save the consolidated data to a CSV file
consolidated_data.to_csv("data/consolidated_tyre_data_with_tire_info.csv", index=False)


      driverId  raceId          Driver           GP  year      location   
0          1.0     860  Lewis Hamilton    Australia  2012      Adelaide  \
1          1.0     861  Lewis Hamilton     Malaysia  2012  Kuala Lumpur   
2          1.0     862  Lewis Hamilton        China  2012      Shanghai   
3          1.0     863  Lewis Hamilton      Bahrain  2012        Sakhir   
4          1.0     864  Lewis Hamilton        Spain  2012     Barcelona   
...        ...     ...             ...          ...   ...           ...   
4030     858.0    1105  Logan Sargeant        Spain  2023     Barcelona   
4031     858.0    1106  Logan Sargeant       Canada  2023        Quebec   
4032     858.0    1109  Logan Sargeant      Hungary  2023      Budapest   
4033     858.0    1110  Logan Sargeant      Belgium  2023      Brussels   
4034     858.0    1111  Logan Sargeant  Netherlands  2023     Zandvoort   

     driverRef  Total Laps                                          Tire Info  
0     hamilton     

In [ ]:
pit_stops_df.head(20)

In [ ]:
# chat gpt free consolidation




# Consolidate the data based on matches between driverId and raceId
consolidated_data = tyre_data_ID_df.groupby(['driverId', 'raceId']).agg({
    'Driver': 'first',
    'GP': 'first',
    'Tyres': 'first',
    'From': 'min',
    'To': 'max',
    '#Laps': 'sum',
    'year': 'first',
    'location': 'first',
    'driverRef': 'first'
}).reset_index()

# Print the consolidated data
print(consolidated_data)


      driverId  raceId          Driver             GP             Tyres  From   
0          1.0     860  Lewis Hamilton      Australia         Soft Used     1  \
1          1.0     861  Lewis Hamilton       Malaysia  Intermediate New     1   
2          1.0     862  Lewis Hamilton          China         Soft Used     1   
3          1.0     863  Lewis Hamilton        Bahrain         Soft Used     1   
4          1.0     864  Lewis Hamilton          Spain         Soft Used     1   
...        ...     ...             ...            ...               ...   ...   
4533     858.0    1106  Logan Sargeant         Canada        Medium New     1   
4534     858.0    1108  Logan Sargeant  Great Britain          Soft New     1   
4535     858.0    1109  Logan Sargeant        Hungary        Medium New     1   
4536     858.0    1110  Logan Sargeant        Belgium          Soft New     1   
4537     858.0    1111  Logan Sargeant    Netherlands          Soft New     1   

      To  #Laps  year      

In [ ]:
consolidated_data.head(30)

In [ ]:
# show lap number for each tyre type and when changed

# Split the data into two DataFrames: one for Soft Used tires and one for Medium New tires
soft_used_df = tyre_data_ID_df[tyre_data_ID_df['Tyres'] == 'Soft Used'].copy()
medium_new_df = tyre_data_ID_df[tyre_data_ID_df['Tyres'] == 'Medium New'].copy()

# Rename columns in each DataFrame for clarity
soft_used_df.rename(columns={'From': 'Soft Used From', 'To': 'Soft Used To', '#Laps': 'Soft Used Laps'}, inplace=True)
medium_new_df.rename(columns={'From': 'Medium New From', 'To': 'Medium New To', '#Laps': 'Medium New Laps'}, inplace=True)

# Merge the DataFrames back together based on driverId, raceId, and other shared columns
consolidated_data = pd.merge(soft_used_df, medium_new_df, on=['driverId', 'raceId', 'Driver', 'GP', 'year', 'location', 'driverRef'], how='outer')

# Fill missing values with 0 for laps when a driver did not use a particular tire type
consolidated_data.fillna({'Soft Used From': 0, 'Soft Used To': 0, 'Soft Used Laps': 0, 'Medium New From': 0, 'Medium New To': 0, 'Medium New Laps': 0}, inplace=True)

# Print the consolidated data
print(consolidated_data)


      Unnamed: 0_x          Driver             GP    Tyres_x  Soft Used From   
0              6.0  Lewis Hamilton      Australia  Soft Used             1.0  \
1              6.0  Lewis Hamilton      Australia  Soft Used             1.0   
2            167.0  Lewis Hamilton          China  Soft Used             1.0   
3            167.0  Lewis Hamilton          China  Soft Used             1.0   
4            168.0  Lewis Hamilton          China  Soft Used            11.0   
...            ...             ...            ...        ...             ...   
3896           NaN   Nyck de Vries  Great Britain        NaN             0.0   
3897           NaN     Zhou Guanyu  Great Britain        NaN             0.0   
3898           NaN     Zhou Guanyu        Belgium        NaN             0.0   
3899           NaN     Zhou Guanyu    Netherlands        NaN             0.0   
3900           NaN     Liam Lawson    Netherlands        NaN             0.0   

      Soft Used To  Soft Used Laps  yea

In [ ]:
consolidated_data.head(30)

In [ ]:
consolidated_data.to_csv("data/tyres_lapsNr_with_TyreType.csv")

In [ ]:
consolidated_data.head(20)

In [ ]:
tyre_data_ID_df.head(10)

In [ ]:
import pandas as pd

# Assuming you have your data in a DataFrame named 'tyre_data_ID_df'

# Create a new DataFrame to store the consolidated data
consolidated_df = pd.DataFrame()

# Iterate through unique raceId and driverId combinations
for race_id, driver_id in tyre_data_ID_df[['raceId', 'driverId']].drop_duplicates().values:
    race_driver_df = tyre_data_ID_df[(tyre_data_ID_df['raceId'] == race_id) & (tyre_data_ID_df['driverId'] == driver_id)]
    
    # Create a temporary DataFrame to store pit stop data for this combination
    temp_df = pd.DataFrame()
    temp_df['stop'] = race_driver_df['#Laps']
    temp_df['From'] = race_driver_df['From']
    temp_df['To'] = race_driver_df['To']
    temp_df['Tyres'] = race_driver_df['Tyres']
    
    # Concatenate the temporary DataFrame to the consolidated DataFrame
    consolidated_df = pd.concat([consolidated_df, temp_df], ignore_index=True)

# Add the 'pit_stop' column to the original DataFrame
tyre_data_ID_df['pit_stop'] = consolidated_df.values.tolist()

# Display the updated DataFrame
print(tyre_data_ID_df)



ValueError: Length of values (13164) does not match length of index (13302)

In [ ]:


# Create a new DataFrame to store the consolidated data
consolidated_df = pd.DataFrame()

# Create a list to store problematic combinations
problematic_combinations = []

# Iterate through unique raceId and driverId combinations
for race_id, driver_id in tyre_data_ID_df[['raceId', 'driverId']].drop_duplicates().values:
    race_driver_df = tyre_data_ID_df[(tyre_data_ID_df['raceId'] == race_id) & (tyre_data_ID_df['driverId'] == driver_id)]
    
    if len(race_driver_df) != race_driver_df['#Laps'].max():
        # Append problematic combination to the list
        problematic_combinations.append((race_id, driver_id))
    
    # Create a temporary DataFrame to store pit stop data for this combination
    temp_df = pd.DataFrame()
    temp_df['stop'] = race_driver_df['#Laps']
    temp_df['From'] = race_driver_df['From']
    temp_df['To'] = race_driver_df['To']
    temp_df['Tyres'] = race_driver_df['Tyres']
    
    # Concatenate the temporary DataFrame to the consolidated DataFrame
    consolidated_df = pd.concat([consolidated_df, temp_df], ignore_index=True)

# Check problematic combinations
print("Problematic Combinations:")
for race_id, driver_id in problematic_combinations:
    print(f"raceId: {race_id}, driverId: {driver_id}")

# Add the 'pit_stop' column to the original DataFrame
tyre_data_ID_df['pit_stop'] = consolidated_df.values.tolist()

# Display the updated DataFrame
print(tyre_data_ID_df)


Problematic Combinations:
raceId: 860.0, driverId: 1.0
raceId: 861.0, driverId: 1.0
raceId: 862.0, driverId: 1.0
raceId: 863.0, driverId: 1.0
raceId: 864.0, driverId: 1.0
raceId: 865.0, driverId: 1.0
raceId: 866.0, driverId: 1.0
raceId: 867.0, driverId: 1.0
raceId: 868.0, driverId: 1.0
raceId: 869.0, driverId: 1.0
raceId: 870.0, driverId: 1.0
raceId: 872.0, driverId: 1.0
raceId: 873.0, driverId: 1.0
raceId: 874.0, driverId: 1.0
raceId: 875.0, driverId: 1.0
raceId: 876.0, driverId: 1.0
raceId: 877.0, driverId: 1.0
raceId: 878.0, driverId: 1.0
raceId: 879.0, driverId: 1.0
raceId: 880.0, driverId: 1.0
raceId: 881.0, driverId: 1.0
raceId: 882.0, driverId: 1.0
raceId: 883.0, driverId: 1.0
raceId: 884.0, driverId: 1.0
raceId: 885.0, driverId: 1.0
raceId: 886.0, driverId: 1.0
raceId: 887.0, driverId: 1.0
raceId: 888.0, driverId: 1.0
raceId: 890.0, driverId: 1.0
raceId: 891.0, driverId: 1.0
raceId: 892.0, driverId: 1.0
raceId: 893.0, driverId: 1.0
raceId: 894.0, driverId: 1.0
raceId: 895.0, dr

ValueError: Length of values (13164) does not match length of index (13302)

In [ ]:
# Filter out rows with problematic combinations
tyre_data_ID_df_filtered = tyre_data_ID_df[~tyre_data_ID_df[['raceId', 'driverId']].apply(lambda x: tuple(x) in problematic_combinations, axis=1)]


In [ ]:
tyre_data_ID_df_filtered.head(30)